In [3]:
import numpy as np
#import os
import re
import subprocess
from subprocess import call
#import sys
#import numpy.matlib
import scipy.io
from scipy.signal import medfilt
import librosa
import os
from os.path import exists
## Vocoder function definition #########################################################################################
# from typical_FA_contextfeats import contextFeats
# from myfunctions import spectral_selection,  temporal_corr#,temp_vec_corr
# from myfunctions import spectral_corr,statFunctions_Syl,statFunctions_Vwl,smooth,get_labels,vocoder_func


In [5]:
import wave
import struct
import math
from scipy.signal import butter, lfilter

In [4]:
#Auxilliary functions for feature computation
def spectral_selection(x, n):                              #out of the 19 subband energies computed, this function selects the n energies with the highest values
    y = x.shape
    row = y[0]
    col = y[1]
    xx = []
    for i in range(0,col,1):
        v = x[:,i]                                           # his line selects the i-th column of x and assigns it to the variable v
        v = np.array([v])
        v = v.T                 
        t = np.array(np.arange(1,row+1)).reshape(-1,1)  #This line generates a column vector t containing values from 1 to row, representing the row numbers.

        v = np.hstack((v, t))                      #This line horizontally stacks v and t, resulting in a matrix where the first column contains the values of v and the second column contains the row numbers.
        v_sort = v[v[:,0].argsort(),]                 #This line sorts v based on the values in the first column, resulting in v_sort.
        v_sort_sel = v_sort[row-n:row, :]             #This line selects the last n rows from v_sort and assigns the result to v_sort_sel.
        vv = v_sort_sel[v_sort_sel[:,1].argsort(),]   #This line sorts v_sort_sel based on the values in the second column, resulting in vv
        #tt = numpy.array([vv[:,0]])                  #The subsequent code block handles the concatenation of vv[:, 0] (the first column of vv) with the previous iterations' results stored in xx
        if i!=0:
            if i==1:
                pp = np.array([xx])
                pp = pp.T
            else:
                pp = xx
            pp2 = np.array([vv[:,0]])
            pp2 = pp2.T
            xx = np.hstack((pp, pp2))
        else:
            xx = np.concatenate((xx, vv[:,0]))
    return xx                                         # xx is a matrix containing the selected energies for all the frames with size n x col


def temp_vec_corr(x2, t_sigma):
    from scipy.stats import norm
    y = x2.shape
    row = y[0]
    col = y[1]
    wn = norm.pdf(np.arange(1,col+1,1), (col+1)/2, t_sigma)              # pdf function with mean = (col+1)/2 and std = t_sigma as window
    # NOTE: if we use continue to manipulate the variable x2, (the function argument), then it gets reflected back in
    # in the parent function. (No idea why). So create a copy of x2 and work with that.
    x3 = np.zeros((row,col))
    for i in range(0,row,1):
        x3[i,:] = np.multiply(x2[i,:],wn)                                # windowing the frame energies
    s=0
    for i in range(0,col-1,1):
        for j in range(i+1,col,1):
            s+= np.multiply(x3[:,i], x3[:,j])                             # computing the correlation between the consecutive frames 
    if col!=1:
        s = np.sqrt(np.divide(s, (col-1)*col/2))
    else:
        s = x3                                                             
    return s

def temporal_corr(x, win, t_sigma):
    hwin = (win-1)/2           # hwin is the half window size
    yy = x.shape
    row = yy[0]
    col = yy[1]

    row = int(row)
    hwin = int(hwin)

    x = np.array([np.concatenate((np.zeros((row,hwin)), x, np.zeros((row, hwin))), axis = 1)])              # zero padding the input matrix, hwin zeros on each sides of the columns
    y = []
    for i in range(hwin,col+hwin,1):
        temp2 = x[0,:,i-hwin:i+hwin+1]
        z = temp_vec_corr(temp2, t_sigma)
        z = np.array([z]).T
        if i==hwin:
            y = np.concatenate((y, z[:,0]))
        else:
            if i==hwin+1:
                y = np.array([y]).T
            y = np.hstack((y, z))
    return y

def spectral_corr(x):
    yy = x.shape
    row = yy[0]
    col = yy[1]

    s = np.zeros((1, col))
    for i in range(0, row-1, 1):
        for j in range(i+1, row, 1):
            s = s+np.multiply(x[i,:], x[j,:])

    if row!=1:
        s = np.sqrt(np.divide(s, (row*(row-1)/2)))
    else:
        s = x
    return s

def statFunctions_Syl(t):
    from scipy.stats.mstats import gmean
    if np.min(t)<0:
        t = np.subtract(t,min(t[0]))
        #out = []
        #return out
    out = np.array([np.median(t[0]), np.mean(t[0]), gmean(np.absolute(t[0])), np.max(t[0])-np.min(t[0]), np.std(t[0])])
    out = np.array([out]).T
    t = np.subtract(t,np.min(t[0]))
    t = np.divide(t, np.sum(t[0]))
    tempArr = np.array([np.arange(1,len(t[0])+1)])
    temporalMean = np.sum(np.multiply(tempArr,t)[0])
    temporalStd = np.sqrt(np.sum(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),2),t[0])))
    temporalSkewness = np.sum(np.divide(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),3),t[0]),np.power(temporalStd,3)))
    temporalKurthosis = np.sum(np.divide(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),4),t[0]),np.power(temporalStd,4)))
    arr1 = np.array([np.array([temporalStd, temporalSkewness, temporalKurthosis])]).T
    out = np.vstack((out,arr1))
    return out

def statFunctions_Vwl(t):
    if np.min(t)<0:
        t = np.subtract(t,min(t[0]))
        #out = []
        #eturn out
    out = np.array([np.median(t[0]), np.mean(t[0]), np.max(t[0])-np.min(t[0]), np.std(t[0])])
    out = np.array([out]).T
    t = np.subtract(t,np.min(t[0]))
    t = np.divide(t, np.sum(t[0]))
    tempArr = np.array([np.arange(1,len(t[0])+1)])
    temporalMean = np.sum(np.multiply(tempArr,t)[0])
    temporalStd = np.sqrt(np.sum(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),2),t[0])))
    temporalSkewness = np.sum(np.divide(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),3),t[0]),np.power(temporalStd,3)))
    temporalKurthosis = np.sum(np.divide(np.multiply(np.power(np.subtract(np.array([np.arange(1,len(t[0])+1)]),temporalMean),4),t[0]),np.power(temporalStd,4)))
    arr1 = np.array([np.array([temporalStd, temporalSkewness, temporalKurthosis])]).T
    out = np.vstack((out,arr1))
    return out

def smooth(t_cor, swin, sigma):
    from scipy.stats import norm
    ft = norm.pdf(np.arange(1,swin+1), (swin+1)/2, sigma)
    ft = np.array([ft])
    t_cor = np.array([t_cor])
    convRes = np.zeros((1, t_cor.shape[2]+ft.shape[1]-1))
    convRes = np.convolve(t_cor[0,0,:], ft[0,:])
    y = convRes[np.arange((swin+1)//2-1, len(convRes)-(swin-1)//2, 1)]
    return y

def get_labels(lab_list,fa,fileName):
        L=[]; fb=fa; filenm=[];
        
        for num in range(0,len(lab_list)):
            if str((lab_list[num][0].tolist())[0]) == str('P'):
                L.append(1)
                filenm.append(fileName)           
            else:
                L.append(0)
                filenm.append(fileName)
        fb = np.vstack((fa,L))
#        fb = np.vstack((fb,np.asarray(filenm,object)))
        return fb,filenm

def get_labels_seq2seq(lab_list):
        L=[];# filenm=[];
        
        for num in range(0,len(lab_list)):
            if str((lab_list[num][0].tolist())[0]) == str('P'):
                L.append(1)
#                filenm.append(fileName)           
            else:
                L.append(0)
                #filenm.append(fileName)
        #fb = np.vstack((fa,L))
#        fb = np.vstack((fb,np.asarray(filenm,object)))
        return L
    
def vocoder_func(wavPath):

    # FILTER DEFINITIONS

    def butter_bandpass(lowcut, highcut, fs, order):
        nyq = 0.5*fs
        low = float(lowcut) / nyq
        high = float(highcut) / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a

    def butter_lowpass(lowcut, fs, order):
        nyq = 0.5*fs
        low = float(lowcut) / nyq
        b ,a = butter(order, low, btype='lowpass')
        return b, a

    def butter_bandpass_filter(data, lowcut, highcut, fs, order):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

    def butter_lowpass_filter(data, lowcut, fs, order):
        b, a = butter_lowpass(lowcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

    # FUNCTION TO READ A .wav FILE MATLAB STYLE

    def readWav(wavPath):
        waveFile = wave.open(wavPath)
        fs = waveFile.getframerate()
        length = waveFile.getnframes()
        data = []
        for i in range(0, length):
            waveData = waveFile.readframes(1)
            data.append(struct.unpack("<h", waveData))
        waveFile.close()
        data = np.array([data])
        data = data.astype(float)/np.max(np.abs(data))
        data = data[0]
        return data, fs, length

    # BUFFER FUNCTION AS DEFINED IN MATLAB

    def buffer(x, n, p=0, opt=None):
        import numpy
        if p >= n:
            raise ValueError('p ({}) must be less than n ({}).'.format(p,n))
        cols = int(numpy.ceil(len(x)/float(n-p)))+1
        if opt == 'nodelay':
            cols += 1
        elif opt != None:
            raise SystemError('Only `None` (default initial condition) and '
                              '`nodelay` (skip initial condition) have been '
                              'implemented')
        b = numpy.zeros((n, cols))
        j = 0
        for i in range(cols):
            if i == 0 and opt == 'nodelay':
                b[0:n,i] = x[0:n]
                continue
            elif i != 0 and p != 0:
                b[:p, i] = b[-p:, i-1]
            else:
                b[:p, i] = 0
            k = j + n - p
            n_end = p+len(x[j:k])
            b[p:n_end,i] = x[j:k,0]
            j = k
        return b

    fltcF= np.array([240,360,480,600,720,840,1000,1150,1300,1450,1600,1800,2000,2200,2400,2700,3000,3300,3750])
    fltBW= np.array([120,120,120,120,120,120,150,150,150,150,150,200,200,200,200,300,300,300,500])

    fltFc= np.array([np.subtract(fltcF,np.divide(fltBW,2)),np.add(fltcF,np.divide(fltBW,2))])
    fltLpFc= 50

    sig, Fs, length = readWav(wavPath)
    print("sig.shape: ", sig.shape)
    print("Fs: ", Fs)

    # Saving the audio in a txt file
    xx = np.append(Fs,sig)                       # sig is the amplitude of the audio signal

    nWndw = int(round(Fs*0.02))
    print("nWndw is: ", nWndw)

    nOverlap = int(round(Fs*0.01))
    print("nOverlap is: ", nOverlap)

    sig = 0.99*sig/max(abs(sig))                 # Normalizing the signal
    
    # Windowing first and filtering next
    sigFrames= buffer(sig*32768,nWndw,nOverlap)          # sig Frames is a 2D array where each column represents an analysis frame 
    subBandEnergies= np.zeros([19,sigFrames.shape[1]])   # 2D array with 19 rows and number of columns equal to number of frames

    for j in range(0,sigFrames.shape[1]): 
        currFrame = np.array([sigFrames[:,j]])                  # 1D array with the selected frame
        for i in range(0,fltFc.shape[1]):
            fltFrame = butter_bandpass_filter(currFrame[0], fltFc[0][i], fltFc[1][i], Fs, 2); fltFrame = fltFrame.T  # this line applies the bandpass filter to the current frame with fltFc[0][i] as lowcut and fltFc[1][i] as highcut
            rectFrame = np.abs(fltFrame[0:nWndw])
            lpFltFrame = butter_lowpass_filter(rectFrame, float(fltLpFc), Fs, 2)
            currEnergy = lpFltFrame[nWndw-1]
            if currEnergy < 1:
                currEnergy = 0.5
            subBandEnergies[i,j] = math.exp(2*math.log(currEnergy)/math.log(10))
    subBandEnergies = np.concatenate((np.exp(0.5*np.ones((19,1))),subBandEnergies[:,0:-2]),axis=1).T

    return Fs, subBandEnergies, xx

In [6]:
"""
Created on Mon Dec 12 10:51:41 2022

@author: iiit
"""
from typical_FA_contextfeats import contextFeats

ModuleNotFoundError: No module named 'typical_FA_contextfeats'

In [13]:
import tempfile

In [22]:
transDir= "../data/"
dictDir= "../"
transFile = "ISLEtrans.txt"
dictName = "nativeEnglishDict_gt100_manoj.syl"
models = ['fisher-2000_FA_GT_ESTphnTrans_estStress']#'libri-960_FA_GT_ESTphnTrans_estStress','wsj_FA_GT_ESTphnTrans_estStress','fisher-2000_FA_GT_ESTphnTrans_estStress','fisher-30_FA_GT_ESTphnTrans_estStress']
#libri-30_FA_GT_ESTphnTrans_estStress
model_FA = ['FA_htkCorrectedLabWithFullAudio']
labels_mis_count=[]
for mod in range(0,len(models)):
    Features = []
    phonecountError=1
    filenotExist=1
    prevSuccessError=1
    done=0
    label_mismatch=0
    model = models[mod]
    filepath = '../data/'+model+'/lab/txt/phn/'
    files = os.listdir('../data/GER/train/') 
    print(files[0])
    
    wavPath = '../data/GER/train/'
    
    stressLabelspath = '../data/'+model_FA[0]+'/lab/mat/sylStress/'
    #VOWEL LIST
    vowelList= ['aa','ae','ah','ao','aw','ay','eh','er','ey','ih','iy','ow','oy','uh','uw']
    
    for fileN in range(0,len(files)): #len(files)
        
        is_looping = True
        wavFile =wavPath+files[fileN]#files[fileN]
        fileName = files[fileN]#files[fileN]
        phnFile = filepath+fileName[0:-4]+'.txt'
        print(phnFile)
    
        if os.path.exists(stressLabelspath + fileName[:-4] + '.mat'):

            with tempfile.NamedTemporaryFile(mode='w', delete=False) as tmp_file:
                temp_filename = tmp_file.name
                
                # loading of data
                subprocess.call(["type " + phnFile + " | tr \"\t\" \" \" > " + temp_filename], shell=True)

                try:
                    with open(temp_filename, 'r') as fid:
                        dataArray = np.loadtxt(fid, dtype={'names': ('a', 'b', 'c'), 'formats': ('f4', 'f4', 'S16')})

                except FileNotFoundError:
                    print('File does not exist')
                    filenotExist += 1
                    os.remove(temp_filename)  # Clean up temporary file

            os.remove(temp_filename)  # Clean up temporary file after processing
    
            phnTimes1 = [row[0] for row in dataArray];phnTimes1 = np.array([phnTimes1]).T
            phnTimes2 = [row[1] for row in dataArray];phnTimes2 = np.array([phnTimes2]).T
            phnTimes = np.hstack((phnTimes1, phnTimes2))
            phones = [row[2] for row in dataArray];phones = np.array([phones])
            for kk in range (0,len(phones[0])):
                phones[0][kk] = phones[0][kk].lower()      # Made them lowercase since the syl dictionary is in lowercase
            origPhones = phones
            index = np.argwhere(origPhones[0]=='sil')
            phones = phones[phones!='sil']
            phones = np.array([phones])
        
            #Getting vowel data
            vowelStartTime = []; vowelEndTime = []; vowel = []
            for kk in range(0,len(dataArray)):
                if dataArray[kk][2].lower() in vowelList:
                    vowelStartTime.append(dataArray[kk][0]); vowelEndTime.append(dataArray[kk][1]); vowel.append(dataArray[kk][2].lower())
            vowelStartTime = np.array([vowelStartTime])
            vowelEndTime = np.array([vowelEndTime])
            vowel = np.array([vowel])

            print(vowel)
        
            #sylPhnTimes = phnTimes[index][0]
            phnTimes2 = np.delete(phnTimes2, index, axis=0)
            phnTimes = np.delete(phnTimes, index, axis=0)

            print(phnTimes)

            trans_path = transDir + "ISLEtrans.txt"

            # Read the contents of the transcript file
            with open(trans_path, 'r') as trans_file:
                trans_contents = trans_file.read()

            # Extract the lines containing the specified filename from the transcript
            lines = [line for line in trans_contents.split('\n') if fileName in line]

            # Extract the words from the lines and clean them up
            words = []
            for line in lines:
                _, word_list = line.split(' ', 1)
                words.extend(re.findall(r'\b\w+\b', word_list))
            words = [word.lower() for word in words]

            print(words)


            flag = 0
            word_syls = []
            for iterWord in range(0,len(words),1):
                subprocess.call(["cat "+dictDir+dictName+" | sed -e \"s/^/+/g\" | grep \"+"+words[iterWord]+" \" | cut -d\"=\" -f2 > /tmp/tmp.txt"], shell = True)
                with open("/tmp/tmp.txt", 'r') as fid: 
                    tempArr = [tempArr.rstrip('\n') for tempArr in fid]
                word_syls.append(tempArr)
                sylSuccess_flag = 0
            k=0
        
            for i in range(0,len(word_syls)):
                for j in range(0,len(word_syls[i])):
                    if word_syls[i][j][0]== ' ':
                        word_syls[i][j] = word_syls[i][j].replace(' ', '', 1)

            print(word_syls)
        
            # Finding the syllables path that matches phone transcriptions for each file.
            newSuccessInds_all = []
            newSuccessInds_all2 = []
            prevSuccessInds_all = []
            prevSuccessInds_all.append(0)

            for iterWord in range(0,len(words)):
                currWordSyls = word_syls[iterWord]
                countSuccess = 1
                for iterPrev in range(0,len(prevSuccessInds_all)):
                    prevWordSyls = ""
                    if prevSuccessInds_all[iterPrev] == 0:
                        currPrevSylInds = []
                    else:
                        currPrevSylInds=prevSuccessInds_all[iterPrev]
                        for iterPrevSyls in range(0,len(currPrevSylInds)):
                            temp = word_syls[iterPrevSyls]
                            prevWordSyls = prevWordSyls+temp[currPrevSylInds[iterPrevSyls]]+" "

                    for iterCurr in range(0,len(currWordSyls)):
                        
                        currTestWordSyls = prevWordSyls+currWordSyls[iterCurr]
                        temp2 = currTestWordSyls.replace(' . ',' ')
                        inds = [m.start() for m in re.finditer(' ',temp2)]
                        if len(inds) == 0:
                            inds = [len(temp2)]
                        count = 1
                        temp = []
                        for iterTemp in range(0,len(inds),1):
                            if iterTemp == 0:
                                temp1 = temp2[0:inds[iterTemp]]
                            else:
                                temp1 = temp2[inds[iterTemp-1]+1:inds[iterTemp]]
                            if not((np.unique(temp1) == ' ').any() or (len(temp1)==0)):
                                temp.append(temp1)
                                count = count+1
                        if iterTemp==len(inds)-1 and len(inds)<len(currTestWordSyls):
                            temp1 = temp2[inds[iterTemp]+1:len(temp2)]
                            if not((len(temp1)==0) or (np.unique(temp1) == ' ').any()):
                                temp.append(temp1)
                                count = count+1
                        if iterWord+1==len(words):
                            currPhones = phones[0,0:len(phones[0])]
        
                        else:
                            currPhones = phones[0][0:len(temp)]
                        flag = 1
                        for iterFlag in range(0,len(currPhones),1):
                            if len(currPhones)!=len(temp):
                                flag = 0
                            else:
                                if currPhones[iterFlag]!=temp[iterFlag]:
                                    flag = 0
                        if flag==1:
                            if not currPrevSylInds==[]:
                                for i in range(0,len(currPrevSylInds)):
        #                            print('line 122::::::yes')
                                    newSuccessInds_all.append(currPrevSylInds[i])
                            newSuccessInds_all.append(iterCurr)
                            newSuccessInds_all2.append(newSuccessInds_all)
                            newSuccessInds_all = []
                            countSuccess = countSuccess+1
                prevSuccessInds_all = newSuccessInds_all2
                newSuccessInds_all2 = []
        
            # Compute syllable and word times
            if len(prevSuccessInds_all) != 1:
                prevSuccessError = prevSuccessError+1
                continue
            else:
                pathInds= prevSuccessInds_all[0]
                sylCount= 1
                phnCount= 1
                spurtSyl= [] #spurtSylTimes= np.zeros((len(phnTimes),2))
        
                syls_word = np.zeros((1,len(pathInds)))
                spurtWordTimes= np.zeros((len(pathInds),2))

                for iterPath in range(0,len(pathInds)):
                    currWord = words[iterPath]
                    currWordSyls = word_syls[iterPath]
                    currSyl = currWordSyls[pathInds[iterPath]]
                    currSyl= currSyl.replace(' . ','.')
                    inds = [m.start() for m in re.finditer('\.',currSyl)]
                    if len(inds) == 0:
                        inds = [len(currSyl)]
                    count = 0
                    for iterTemp in range(0,len(inds)):
                        if iterTemp == 0:
                            temp1 = currSyl[0:inds[iterTemp]]
                        else:
                            temp1 = currSyl[inds[iterTemp-1]+1:inds[iterTemp]]
                        if not (temp1==' ' or len(temp1) == 0):
                            spurtSyl.append(temp1)
                            sylCount = sylCount+1
                            count = count+1
                    if iterTemp is len(inds)-1 and len(inds)<len(currTestWordSyls):
                        temp1 = currSyl[inds[iterTemp]+1:len(currSyl)]
                        if not (temp1==' ' or len(temp1) == 0):
                            spurtSyl.append(temp1)
                            sylCount = sylCount+1
                            count = count+1
                    syls_word[0][iterPath] = count
        
                spurtSylTimes = np.zeros((len(spurtSyl),2))
        
                for iterSyl in range(0,len(spurtSyl)):
                    temp2 = spurtSyl[iterSyl]
                    inds = [m.start() for m in re.finditer(' ',temp2)]
                    if len(inds) == 0:
                        inds = [len(temp2)]
                    count = 1
                    temp = []
                    for iterTemp in range(0,len(inds)):
                        if iterTemp == 0:
                            temp1 = temp2[0:inds[iterTemp]]
                        else:
                            temp1 = temp2[inds[iterTemp-1]+1:inds[iterTemp]]
                        if not(temp1 == ' ' or len(temp1) == 0):
                            temp.append(temp1)
                            count = count+1
                    if iterTemp==len(inds)-1 and len(inds)<len(currTestWordSyls):
                        temp1 = temp2[inds[iterTemp]+1:len(temp2)]
                        if not (temp1 == ' ' or len(temp1)==0):
                            temp.append(temp1)
                            count = count+1
        
                    nPhns_syl = len(temp)
                    spurtSylTimes[iterSyl,0] = phnTimes[phnCount-1,0]
                    phnCount = phnCount + nPhns_syl
                    spurtSylTimes[iterSyl,1] = phnTimes[phnCount-1-1,1]
                length_spurtSylTimes = iterSyl+1
                if len(phones[0]) != (phnCount-1):
                    phonecountError = phonecountError+1
        
                sylIdx = 1
                for iterWordTimes in range(0,len(syls_word[0])):
                    spurtWordTimes[iterWordTimes,0] = spurtSylTimes[sylIdx-1,0]
                    sylIdx = sylIdx + syls_word[0][iterWordTimes].astype(int)
                    spurtWordTimes[iterWordTimes,1] = spurtSylTimes[sylIdx-1-1,1]
                length_spurtWordTimes = iterWordTimes+1
        
        
        # Create syllable files done    ########################################################################################
                                        ########################################################################################
        # Compute features              ########################################################################################
            twin = 5
            t_sigma = 1.4
            swin = 7
            s_sigma = 1.5
            mwin = 13
            max_threshold = 25
            
            vwlSB_num= 4
            vowelSB= [1,2,4,5,6,7,8,13,14,15,16,17]
            sylSB_num= 5
            sylSB= [1,2,3,4,5,6,13,14,15,16,17,18]
            
            startWordFrame_all = []
            spurtStartFrame_all = []
            spurtEndFrame_all=[]
            vowelStartFrame_all = []
            vowelEndFrame_all = []
            eng_full_all = []
            spurtStress_all = []
            
            # Execute the vocoder [MODIFICATION]: Get the audio file back so that it can be stored in a text file for C code.
            Fs, eng_full, xx = vocoder_func(wavFile)
            #eng_full = np.loadtxt('./ISLE_SESS0003_BLOCKD01_11_sprt1.e19' , delimiter=',')
            eng_full = eng_full.conj().transpose()
            
            
            # Processing word boundary file
            # FILE READ DELETED HERE
            a = spurtWordTimes
            b = words
            if(len(a) is not len(b)):
                continue
            wordData = np.hstack((a, np.array([b], dtype='S32').T))
            startWordTime = [row[0] for row in wordData]  # Extract first coloumn of wordData
            endWordTime = [row[1] for row in wordData]
            startWordFrame = np.round((np.subtract(np.array(startWordTime, dtype='float'), spurtSylTimes[0][0].astype(float))*100))
            endWordFrame = np.round((np.subtract(np.array(endWordTime, dtype='float'), spurtSylTimes[0][0].astype(float))*100) + 1)
            startWordFrame = np.append(startWordFrame,endWordFrame[-1])
            
            # Processing of stress and syllable boundary file
            spurtSylTime = spurtSylTimes
            spurtStartTime = spurtSylTime[:, 0]
            spurtEndTime = spurtSylTime[:, 1]
            spurtStartFrame = np.round((spurtStartTime - spurtStartTime[0]) * 100)
            spurtEndFrame = np.round((spurtEndTime - spurtStartTime[0]) * 100)
            
            # Processing of Vowel boundary file
            vowelStartFrame = np.round(vowelStartTime*100 - spurtStartTime[0] * 100)
            vowelEndFrame = np.round(vowelEndTime*100 - spurtStartTime[0] * 100)
            
            # TCSSBC computation
            if len(sylSB) > sylSB_num:
                eng = spectral_selection(eng_full[np.subtract(sylSB, 1), :], sylSB_num)
            else:
                eng = eng_full[sylSB, :]
            t_cor = temporal_corr(eng, twin, t_sigma)
            s_cor = spectral_corr(t_cor)
            sylTCSSBC = smooth(s_cor, swin, s_sigma)
            sylTCSSBC = np.array([sylTCSSBC])
            
            # Modify TCSSBC contour by clipping from the syllable start
            start_idx = np.round(spurtStartTime[0]*100).astype(int)
            sylTCSSBC = np.array([sylTCSSBC[0][start_idx:-1]])
            
            sylTCSSBC = np.divide(sylTCSSBC, max(sylTCSSBC[0]))
            
            if len(vowelSB) > vwlSB_num:
                eng = spectral_selection(eng_full[np.subtract(vowelSB, 1), :], vwlSB_num)
            else:
                eng = eng_full[vowelSB, :]
            t_cor = temporal_corr(eng, twin, t_sigma)
            s_cor = spectral_corr(t_cor)
            vwlTCSSBC = smooth(s_cor, swin, s_sigma)
            
            vwlTCSSBC = np.array([vwlTCSSBC])
            
            # Modify TCSSBC contour by clipping from the vowel start
            start_idx = np.round(vowelStartTime[0][0]*100).astype(int)
            vwlTCSSBC = np.array([vwlTCSSBC[0][start_idx:-1]])
            
            vwlTCSSBC = np.divide(vwlTCSSBC, max(vwlTCSSBC[0]))
            
            # Compute silence statistics
            # Preprocessing of the data
            word_duration = np.zeros((1, len(startWordFrame) - 1))
            word_Sylsum = np.zeros((1, len(startWordFrame) - 1))
            word_Vwlsum = np.zeros((1, len(startWordFrame) - 1))
            
            for j in range(0, len(startWordFrame) - 1):
                temp_start = startWordFrame[j].astype(int)
                temp_end = startWordFrame[j + 1].astype(int) - 1
                #jhansi
                if (temp_end >= sylTCSSBC.shape[1]):
                    temp_end1 = sylTCSSBC.shape[1]-1
                    sylTCSSBC[0, np.arange(temp_start, temp_end1)] = medfilt(sylTCSSBC[0, np.arange(temp_start, temp_end1)], 3)
                    sylTCSSBC[0, temp_start] = sylTCSSBC[0, temp_start+1]
                    sylTCSSBC[0, temp_end1] = sylTCSSBC[0, temp_end1 - 1]
                    tempArr = sylTCSSBC[0, np.arange(temp_start, temp_end1)]
                    word_Sylsum[0, j] = tempArr.sum(axis=0)
                else:
                    sylTCSSBC[0, np.arange(temp_start, temp_end)] = medfilt(sylTCSSBC[0, np.arange(temp_start, temp_end)], 3)
                    sylTCSSBC[0, temp_start] = sylTCSSBC[0, temp_start+1]
                    sylTCSSBC[0, temp_end] = sylTCSSBC[0, temp_end - 1]
                    tempArr = sylTCSSBC[0, np.arange(temp_start, temp_end)]
                    word_Sylsum[0, j] = tempArr.sum(axis=0)
                if (temp_end >= vwlTCSSBC.shape[1]):
                    temp_end = vwlTCSSBC.shape[1]-1
            #    temp_end = np.min([temp_end,len(vwlTCSSBC)])
                vwlTCSSBC[0, np.arange(temp_start, temp_end)] = medfilt(vwlTCSSBC[0, np.arange(temp_start, temp_end)], 3)
                vwlTCSSBC[0, temp_start] = vwlTCSSBC[0, temp_start+1]
                vwlTCSSBC[0, temp_end] = vwlTCSSBC[0, temp_end - 1]
            
                word_duration[0, j] = temp_end - temp_start + 1
    
                tempArr = vwlTCSSBC[0, np.arange(temp_start, temp_end)]
                word_Vwlsum[0, j] = tempArr.sum(axis=0)
            sylTCSSBC[np.isnan(sylTCSSBC)] = 0
            vwlTCSSBC[np.isnan(vwlTCSSBC)] = 0
            tempOut = np.array([[]])
            
            wordIndication = []
            peakVals = []
            avgVals = []
            
            # Generating the features
            for j in range(0, len(spurtSyl), 1):
                inds = (startWordFrame <= spurtStartFrame[j]).nonzero()
                word_ind = inds[0][-1]
                wordIndication.append(word_ind)
        #       print([0, np.arange(spurtStartFrame[j], spurtEndFrame[j]-1, 1).astype(int)])
                currFtr1SylSeg = sylTCSSBC[0, np.arange(spurtStartFrame[j], spurtEndFrame[j]-1, 1).astype(int)]
                currFtr1SylSeg = np.array([currFtr1SylSeg])
                temp = np.multiply(currFtr1SylSeg, len(currFtr1SylSeg[0]) / word_duration[0, word_ind])
                # arrResampled = np.array([librosa.resample(temp[0], float(30) / len(temp[0]), 'sinc_best')])
                arrResampled = np.array([librosa.resample(temp[0], Fs, Fs*float(30) / len(temp[0]), 'sinc_best')])       ##change
                F_new = Fs*float(30) / len(temp[0])
                
                #To be put in the output file
                peakVals.append(np.amax(arrResampled))
                avgVals.append(np.average(arrResampled))
            
                currSylFtrs = statFunctions_Syl(arrResampled)
                arr1 = np.array([np.array([np.sum(currFtr1SylSeg) / word_Sylsum[0, word_ind]])]).T
                currSylFtrs = np.vstack((currSylFtrs, arr1))
                ##########jhansi
                if (j>= vowelEndFrame.shape[1]):
                    break
                if (vowelEndFrame [0,j] >= vwlTCSSBC.shape[1]):
                    vowelEndFrame[0,j] = vwlTCSSBC.shape[1]-1
            
                currFtr1VowelSeg = vwlTCSSBC[0, np.arange(vowelStartFrame[0, j], vowelEndFrame[0, j]-1, 1).astype(int)]
                currFtr1VowelSeg = np.array([currFtr1VowelSeg])
                temp = np.multiply(currFtr1VowelSeg, len(currFtr1VowelSeg[0]) / word_duration[0, word_ind])
                if (len(temp[0])==0):
                    break
                    
                #arrResampled = np.array([scikits.samplerate.resample(temp[0], float(20) / len(temp[0]), 'sinc_best')])
                arrResampled = np.array([librosa.resample(temp[0], F_new, F_new*float(20) / len(temp[0]), 'sinc_best')])
                currVowelFtrs = statFunctions_Vwl(arrResampled)
                arr1 = np.array([np.array([np.sum(currFtr1VowelSeg) / word_Sylsum[0, word_ind]])]).T
                currVowelFtrs = np.vstack((currVowelFtrs, arr1))
                if j == 0:
                    tempOut = np.vstack((currSylFtrs, currVowelFtrs, len(currFtr1VowelSeg[0]), len(currFtr1SylSeg[0])))
                else:
                    tempOut = np.hstack((tempOut, np.vstack((currSylFtrs, currVowelFtrs,len(currFtr1VowelSeg[0]), len(currFtr1SylSeg[0])))))
            if (len(temp[0])==0):
                   continue
            sylDurations = spurtEndTime - spurtStartTime
            
            ftrs = tempOut
            
            wordLabls = np.unique(wordIndication)
            for iterWrd in range(0, len(wordLabls)):
                inds = [i for i, x in enumerate(wordIndication) if x == wordLabls[iterWrd]] #doing argwhere(wordIndication==wordLabls[iterWrd]
                if len(inds)>1 :
                    ftrs[-1, inds] = ftrs[-1, inds] / sum(ftrs[-1, inds])
                    ftrs[-2, inds] = ftrs[-2, inds] / sum(ftrs[-2, inds])
            end=1
            print(ftrs.shape)
            fa = ftrs
            
            mat = scipy.io.loadmat(stressLabelspath+fileName[0:-4]+'.mat')
            lab = mat['spurtStress']
            lab_list = lab.tolist()
            if (fa.shape[1] != len(lab_list)):
                label_mismatch = label_mismatch+1
    #            is_looping = False
                continue
            else:
                fb,filenm = get_labels(lab_list,fa,fileName)
                feats = fb #features ,last row:labels
                w=[]#polysyl_feat=[];
                for w_l in range(len(words)):
            #        cou = 0
                    w_st = spurtWordTimes[w_l][0]
                    w_ed = spurtWordTimes[w_l][1]

                    for s_l in range(len(w),len(spurtSyl)):
                        sy_st = spurtSylTimes[s_l][0]
                        sy_ed = spurtSylTimes[s_l][1]

                        if (sy_ed <= w_ed):
                            w.append(w_l+1)
    #                        w.append('W'+str(w_l+1))
            #                cou=cou+1
                        else:
                            break
                if (len(w)>np.shape(feats)[1]):
                    continue
                feats = np.vstack((feats,w))#features ,last row:labels, word labels
                AF_inform = filenm
                CF_feats,CF_inform = contextFeats(spurtSyl,spurtSylTimes,spurtWordTimes,vowel)
                if fileN == 0:#411 or fileN ==412:
                    AF = feats
                    AF_info = AF_inform
                    CF = CF_feats
                    CF_info =CF_inform                
                else:
                    AF = np.hstack((AF,feats)) 
                    AF_info = np.hstack((AF_info,AF_inform))
                    CF = np.hstack((CF,CF_feats))
                    CF_info = np.hstack((CF_info,CF_inform))
                    done=done+1
                    print('Done:::file number ' + str(done) + '  out of ' + str(len(files)))
          
    labels_mis_count.append(label_mismatch)
    # scipy.io.savemat('/home/iiit/Desktop/Jhansi/Stress detection/Codes_jhansi/features/GER_train_'+model+'.mat', {'AF': AF,'CF': CF,'CF_info': CF_info,'AF_info': AF_info})
    print(AF,CF,CF_info,AF_info)


ISLE_SESS0006_BLOCKD01_01_sprt1.wav
../data/fisher-2000_FA_GT_ESTphnTrans_estStress/lab/txt/phn/ISLE_SESS0006_BLOCKD01_01_sprt1.txt
[]
[]
[]
[]


C:\Users\sanke\AppData\Local\Temp\ipykernel_19844\3414640566.py:45: UserWarning: loadtxt: input contained no data: "<_io.TextIOWrapper name='C:\\Users\\sanke\\AppData\\Local\\Temp\\tmpy8uj1dy8' mode='r' encoding='cp1252'>"
  dataArray = np.loadtxt(fid, dtype={'names': ('a', 'b', 'c'), 'formats': ('f4', 'f4', 'S16')})
C:\Users\sanke\AppData\Local\Temp\ipykernel_19844\3414640566.py:61: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  index = np.argwhere(origPhones[0]=='sil')
C:\Users\sanke\AppData\Local\Temp\ipykernel_19844\3414640566.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  phones = phones[phones!='sil']


TypeError: object of type 'int' has no len()